# Data Organizer

A collection of code-snippets to help organize and manage MIAPPE-compliant metadata tables.

In [56]:
# Example to map an ppeo excel file into JSON-LD format

import pandas as pd
import json



PATH = r'/home/gryvity/Desktop/workstation/lab/MIAPPEx/scripts/notebooks/test_data/example_01_v1.1_ext.xlsx'

standard_sheet_names = ['Investigation', 
                        'Datafile', 
                        'Person', 
                        'Study', 
                        'Observation Unit', 
                        'Observed Variable', 
                        'Biological Material', 'Environment', 
                        'Factor', 
                        'Event', 
                        'Sample']




# Load the Excel file sheets into a dictionary of DataFrames
sheets = pd.read_excel(PATH, sheet_name=None).keys()



# Now read for each sheet all column names and sort them into a dictionary
column_names = {}
for sheet in sheets:
    if sheet in standard_sheet_names:
        df = pd.read_excel(PATH, sheet_name=sheet)
        column_names[sheet] = list(df.columns)[1:]


print(json.dumps(column_names, indent=4))





{
    "Investigation": [
        "Investigation unique ID",
        "Investigation title",
        "Investigation description",
        "Submission date",
        "Public release date",
        "License",
        "MIAFPE version",
        "Associated publication",
        "Metadata file version"
    ],
    "Person": [
        "Study unique ID",
        "Person name",
        "Person email",
        "Person ID",
        "Person role",
        "Person affiliation"
    ],
    "Study": [
        "Study unique ID",
        "Study title",
        "Study description",
        "Start date of study",
        "End date of study",
        "Contact institution",
        "Geographic location (country)",
        "Experimental site name",
        "Geographic location (latitude)",
        "Geographic location (longitude)",
        "Geographic location (altitude)",
        "Description of the experimental design",
        "Type of experimental design",
        "Observation unit level hierarchy",
      

In [78]:
import os 
import pandas as pd 
import re
from collections import defaultdict


class MIAPPExDataModel:
    """
    Docstring for MIAPPExDataModel

    Definitions for the DataModel
    """
    def __init__(self):

        self._context = {
            'ppeo' : 'http://purl.org/ppeo/',
            'sosa' : 'http://www.w3.org/ns/sosa/',
            'ssn'  : 'http://www.w3.org/ns/ssn/'
        }

        self._standard_sheet_names = [r'Investigation', 
                                      r'Data*File', 
                                      r'Person', 
                                      r'Study', 
                                      r'Observation*Unit', 
                                      r'Observed*Variable', 
                                      r'Biological*Material', 
                                      r'Environment', 
                                      r'*Factor', 
                                      r'Event', 
                                      r'Sample']
        
        self._sosa_sheet_names = [r'System',
                                  r'Sensor',
                                  r'Platform',
                                  r'Feature*Of*Interest',
                                  r'Procedure']
        
        self.std_sheets = self._standard_sheet_names + self._sosa_sheet_names
        
        self._classes_miappe = {
            'Investigation': 'ppeo:Investigation',
            'DataFile': 'ppeo:Datafile',
            'Person': 'ppeo:Person',
            'Study': 'ppeo:Study',
            'ObservationUnit': 'ppeo:ObservationUnit',
            'ObservedVariable': 'ppeo:ObservedVariable',
            'BiologicalMaterial': 'ppeo:BiologicalMaterial',
            'Environment': 'ppeo:Environment',
            'Factor': 'ppeo:Factor',
            'Event': 'ppeo:Event',
            'Sample': 'ppeo:Sample'
        }
        self._classes_sosassn = {
            'System': 'ssn:System',
            'Sensor': 'sosa:Sensor',
            'Platform': 'sosa:Platform',
            'FeatureOfInterest': 'sosa:FeatureOfInterest',
            'Procedure': 'sosa:Procedure',
            'Input' : 'ssn:Input',
            'Output' : 'ssn:Output'
        }

        self._obj_properties_miappe = {
            'name_01': {
                'context': 'ppeo',
                'accession': '',
                'domain': [],
                'range': [],
            }
        }
        self._obj_properties_sosassn = {}


        self._Individuals = {
            "example_01" : {
                "type": "exampleType", 
                "accession": "http://example.org/individuals/example"
            }
           
        }

    def load_ontology(self, path):
        pass
                                  

class MIAPPExXLSX(MIAPPExDataModel):
    
    """
    Docstring for MIAPPExXLSX

    """     
    def __init__(self, path):
        super().__init__()
        self.path = path
        self.validate_format()

        # Loading Data
        self.data = self.load()
        self.sheets = self.get_sheet_names()
        
        # Get MIAPPE Version
        self.version = self.get_version()

        # Collect Further Structure Information
        self.sheet_info = {}
        
        self.get_properties()
        self.get_sheet_info()

    # Loading File
    def validate_format(self):
        # Validate file format based on extension
        self.format = os.path.splitext(self.path)[-1]
        if self.format != '.xlsx':
            raise ValueError(f'Unsupported file format: {self.format}')
        
    def load(self):
        # Until now only Excel format is supported
        if self.format == '.xlsx':
            return pd.read_excel(self.path, sheet_name=None)
        else:
            raise ValueError(f'Unsupported file format: {self.format}')
        
    # Get the Version
    def get_version(self):
        # Get MIAPPE version from the Investigation sheet
        if 'Investigation' in self.data:
            if 'Field' in self.data['Investigation'].columns:
                return 1.1
            else:
                return 1.2
        else:
            raise ValueError('Investigation sheet not found in the data.')

    

    # Organizing On Sheet Level
    def get_sheet_names(self):
        # Get sheet names from the Excel file and returns a dictionary with standardized names
        sheets = defaultdict(str)
        for sheet_name in self.data.keys():
            stdname =  self.standardize_sheet_name(sheet_name)
            if stdname:
                sheets[sheet_name] = stdname
            else:
                sheets[sheet_name] = 'unknown'  # Mark unknown sheets
        return sheets

    def standardize_sheet_name(self, sheet_name):
        # Standardize sheet names based on predefined patterns
        if 'appendix' in sheet_name.lower():
            return 'appendix'
        elif 'readme' in sheet_name.lower():
            return 'readme'
        for std in self.std_sheets:
            pattern = re.compile(rf'^{std.replace("*", ".*")}$', re.IGNORECASE)
            if pattern.match(sheet_name):
                return std.replace("*", "")
        return False

    # Organizing on Property Level
    def get_properties(self):
        # Get column names for a given sheet
        self.structure = {}
        if self.version == 1.1:
            i = 1
        else:
            i = 0
        for sheet in self.sheets.keys():
            # Only get columns if we have not a readme or appendix sheet
            if self.sheets[sheet] in ['readme', 'appendix']:
                continue
            df = pd.read_excel(self.path, sheet_name=sheet)
            self.structure[sheet] = list(df.columns)[i:]

    def get_sheet_info(self):
        # Gets additional information
        self.sheet_info = {}
        data_pivot = None
        for sheet in self.sheets.keys():
            # Only get columns if we have not a readme or appendix sheet
            if self.sheets[sheet] in ['readme', 'appendix']:
                continue
                

            if self.version == 1.1:
                # Identify the row where the word 'value' appears in the first column
                for i,n in enumerate(self.data[sheet].iloc[:, 0]):
                    if 'value' in str(n).lower():
                        data_pivot = (i, 1)
                        break
            else:
                data_pivot = (0, 0)

           
            self.sheet_info[sheet] = {
                'num_rows': self.data[sheet].shape[0],
                'num_columns': self.data[sheet].shape[1],
                'data_pivot' : data_pivot,
                'data': self.data[sheet].iloc[data_pivot[0]:, data_pivot[1]:].reset_index(drop=True)
            }
            print(f"Sheet: {sheet}, Info: {self.sheet_info[sheet]}")
        


  
        
    

# if __name__ == "__main__":       
base_path = "/home/gryvity/Desktop/workstation/lab/MIAPPEx/scripts/notebooks/"
example_files = [os.path.join(base_path, 'test_data/example_01_v1.1_ext.xlsx'),
                     os.path.join(base_path, 'test_data/example_02_v1.1_ext.xlsx'),
                     os.path.join(base_path, 'test_data/example_01_v1.2.xlsx'),
                     os.path.join(base_path, 'test_data/example_02_v1.2.xlsx')]
    

picked_example = example_files[2]


MIAPPEx_data = MIAPPExXLSX(picked_example)


Sheet: Investigation, Info: {'num_rows': 1, 'num_columns': 9, 'data_pivot': (0, 0), 'data':                    investigationId  \
0  https://doi.org/10.15454/IASSTN   

                                  investigationTitle  \
0  A multi-site experiment in a network of Europe...   

                            investigationDescription submissionDate  \
0  This dataset comes from the European Union pro...     2018-10-10   

  publicReleaseDate       license  miappeVersion  associatedPublication  \
0        2019-03-27  CC BY-SA 4.0            1.2                    NaN   

                                   variablesOntology  
0  The file 12a-Info-EIPO-Environmental_Traits.cs...  }
Sheet: Study, Info: {'num_rows': 19, 'num_columns': 19, 'data_pivot': (0, 0), 'data':    studyId                           studyTitle  \
0    Gai12                Biogemma Gaillac 2012   
1    Gai13                Biogemma Gaillac 2013   
2    Gra13               Biogemma Graneros 2013   
3    Ner11             

In [82]:
import pandas as pd 

example_files = [os.path.join(base_path, 'test_data/example_01_v1.1_ext.xlsx'),
                     os.path.join(base_path, 'test_data/example_02_v1.1_ext.xlsx'),
                     os.path.join(base_path, 'test_data/example_01_v1.2.xlsx'),
                     os.path.join(base_path, 'test_data/example_02_v1.2.xlsx')]
    

picked_example = example_files[2]

# Select Investigation Sheet
df = pd.read_excel(picked_example, sheet_name='Investigation')

# Get Colors of Fonts and Backgrounds
from openpyxl import load_workbook
wb = load_workbook(picked_example)
ws = wb['Investigation']
font_colors = {}
bg_colors = {}
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        font_colors[cell.coordinate] = cell.font.color.rgb if cell.font.color else None
        bg_colors[cell.coordinate] = cell.fill.fgColor.rgb if cell.fill.fgColor else None

bg_colors

{'A1': '00000000',
 'B1': '00000000',
 'C1': '00000000',
 'D1': '00000000',
 'E1': '00000000',
 'F1': '00000000',
 'G1': '00000000',
 'H1': '00000000',
 'I1': '00000000',
 'A2': '00000000',
 'B2': '00000000',
 'C2': '00000000',
 'D2': '00000000',
 'E2': '00000000',
 'F2': '00000000',
 'G2': '00000000',
 'H2': '00000000',
 'I2': '00000000',
 'A3': '00000000',
 'B3': '00000000',
 'C3': '00000000',
 'D3': '00000000',
 'E3': '00000000',
 'F3': '00000000',
 'G3': '00000000',
 'H3': '00000000',
 'I3': '00000000',
 'A4': '00000000',
 'B4': '00000000',
 'C4': '00000000',
 'D4': '00000000',
 'E4': '00000000',
 'F4': '00000000',
 'G4': '00000000',
 'H4': '00000000',
 'I4': '00000000',
 'A5': '00000000',
 'B5': '00000000',
 'C5': '00000000',
 'D5': '00000000',
 'E5': '00000000',
 'F5': '00000000',
 'G5': '00000000',
 'H5': '00000000',
 'I5': '00000000',
 'A6': '00000000',
 'B6': '00000000',
 'C6': '00000000',
 'D6': '00000000',
 'E6': '00000000',
 'F6': '00000000',
 'G6': '00000000',
 'H6': '0000